Experiment: Uses Recansen's Features
------


In [6]:
import sys; sys.path.append('../..')
import torch
from bias_classification import prepare_model 
from src.utils import *
from tasks.bias_classification.lib.tagging.features import Featurizer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
params = prepare_model.intialize_params("experiment_params.json")
dataset = prepare_model.initialize_dataset(params.intermediary_task)
#attention_dataloader = dataset.return_dataloader()
#attention_experiment = prepare_model.initialize_attention_experiment(params.intermediary_task)

06/05/2019 19:06:57 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
440it [00:00, 8016.46it/s]


In [9]:
path_label_data = params.intermediary_task['task_specific_params']['target_data']
all_bert_toks = prepare_model.get_sample_toks(path_label_data)

440it [00:00, 94786.53it/s]


In [12]:
tok2id = prepare_model.get_tok2id(params.intermediary_task)

06/05/2019 19:23:32 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [16]:
featurizer = Featurizer(tok2id, params=params.intermediary_task)

In [33]:
def bert_to_word_tokens(tokens):
    """
    Detokenizes an input tokenized by BertTokenizer.
    @param tokens (list[str])
    @returns sent (str) detokenized str
    """
    bert_to_word = {}
    word_tokens = []
    word_token_idx = 0
    for bert_token_idx, bert_token in enumerate(tokens):
        bert_to_word[bert_token_idx] = word_token_idx
        if bert_token.startswith("##"):
            word_tokens[-1] += bert_token.replace("##", "")
        else:
            word_tokens.append(bert_token)
            word_token_idx += 1
                        
    return word_tokens, bert_to_word

In [59]:
def add_bias_features(dataset, featurizer):
    """
    Adds embeddings 
    @
    """
    intermediary_labels = dataset.get_val('pre_tok_label_ids')
    embeddings = []
    for entry in tqdm(dataset):
        idx = int(entry["index"])
        bert_toks = all_bert_toks[idx]
        word_tokens, bert_to_word = bert_to_word_tokens(bert_toks)
        
        bias_idx = entry['pre_tok_label_ids'].to(dtype=torch.int).flatten().tolist().index(1)
        bias_word = word_tokens[bert_to_word[bias_idx]]
        
        features = featurizer.features(entry["pre_ids"].tolist(), 
                                       entry["rel_ids"].tolist(), 
                                       entry["pos_ids"].tolist())
        embeddings.append(features[bias_idx, :])
        
    tensor = torch.tensor(np.stack(embeddings), dtype=torch.float32)  # num_entries, dim
    dataset.add_data(tensor, "bias_features")

In [60]:
add_bias_features(dataset, featurizer)

Run the cell below to reset params 
-----

In [51]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [12]:
print(params.final_task)

{'model': 'shallow_nn', 'window_size': 5, 'input_dim': 44, 'hidden_dim': 10, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 200, 'batch_size': 32, 'lr': 0.01}}


Boostrapping procedure 

In [ ]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [16]:
statistics

{'auc': [(0.6752843679348468, 0.8509239577269152), 0.7677060136452627],
 'accuracy': [(0.6395348837209303, 0.8023255813953488), 0.7262790697674417]}

New run 

In [35]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [36]:
print(params.final_task)

{'model': 'shallow_nn', 'window_size': 5, 'input_dim': 11, 'hidden_dim': 3, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 200, 'batch_size': 32, 'lr': 0.01}}


Boostrapping procedure 

In [ ]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [22]:
statistics

{'auc': [(0.49685111400227683, 0.8255225075881711), 0.7107909485455589],
 'accuracy': [(0.5869186046511627, 0.7502906976744186), 0.6730232558139536]}

## No bootstrapping -- cannot calculate 95% confidence intervals 

In [61]:
data_split = params.final_task['data_split']
batch_size = params.final_task['training_params']['batch_size']
train_dataloader, eval_dataloader, test_dataloader = dataset.split_train_eval_test(**data_split, batch_size=batch_size)

In [62]:
losses, evaluations = classification_experiment.train_model(train_dataloader, eval_dataloader, input_key='bias_features', label_key='bias_label', threshold=0.42)




In [63]:
avg_losses = [average_data(epoch_losses) for epoch_losses in losses]
avg_predictions = [average_data(epoch_evaluations) for epoch_evaluations in evaluations]

In [64]:
min_loss, max_loss, avg_loss = get_statistics(avg_losses, "loss")
print(min_loss)

0.04847464928853101


In [65]:
min_auc, max_auc, avg_auc = get_statistics(avg_predictions, "auc")

In [66]:
print(max_auc)

0.9656565656565657


In [67]:
print(min_auc)

0.9272150072150072
